In [ ]:
有两种方式：
1. 基于规则（基于语法规则、基于模板）
    原理：基于预先设置好的规则来生成文本（如对话）
    1.1 基于语法规则
    1.2 基于模板
2. 基于概率

In [3]:
# 1.1 基于语法规则

import random

# 简单的语法规则

simple_grammar = """
sentence => noun_phrase verb_phrase
noun_phrase => Article Adj* noun
Adj* => null | Adj Adj*
verb_phrase => verb noun_phrase
Article =>  一个 | 这个 | 那个
noun =>   女孩 | 男孩 | 教室 | 员工 | 电脑 | 眼镜 | 办公室
verb => 看着 | 坐在 |  听着 | 看见 | 用着 | 戴着
Adj =>  温暖的 | 安静的 | 温柔的 | 细心的 | 难过的
"""

# 生成单个形容词
def adj():
    return random.choice("温暖的 | 安静的 | 温柔的 | 细心的 | 难过的".split("|")).split()[0]
# adj()

# 生成0个或多个形容词
def adj_star():
    return random.choice([lambda: '', lambda: adj() + adj_star()])()
adj_star()

'难过的温柔的'

In [4]:
# 将预定义语法字符串转化成 python 格式
def create_grammar(grammar_str, split='=>', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip():
            continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

# 生成语法
adj_grammar = """
Adj* => null | Adj Adj*
Adj => 温暖的 | 安静的 | 温柔的 | 细心的 | 难过的
"""
grammar = create_grammar(adj_grammar)
grammar

{'Adj*': [['null'], ['Adj', 'Adj*']],
 'Adj': [['温暖的'], ['安静的'], ['温柔的'], ['细心的'], ['难过的']]}

In [5]:
# 根据语法生成语句
from random import choice
def generate(gram, target):
    if target not in gram:
        return target
    expanded = [generate(gram, t) for t in choice(gram[target])]
    return ''.join(e for e in expanded if e != 'null')

example_grammar = create_grammar(simple_grammar)
example_grammar

{'sentence': [['noun_phrase', 'verb_phrase']],
 'noun_phrase': [['Article', 'Adj*', 'noun']],
 'Adj*': [['null'], ['Adj', 'Adj*']],
 'verb_phrase': [['verb', 'noun_phrase']],
 'Article': [['一个'], ['这个'], ['那个']],
 'noun': [['女孩'], ['男孩'], ['教室'], ['员工'], ['电脑'], ['眼镜'], ['办公室']],
 'verb': [['看着'], ['坐在'], ['听着'], ['看见'], ['用着'], ['戴着']],
 'Adj': [['温暖的'], ['安静的'], ['温柔的'], ['细心的'], ['难过的']]}

In [6]:
generate(gram=example_grammar, target='sentence')

'那个教室戴着那个细心的教室'

In [28]:
#### 1. 模拟人类和机器人对话：
# 在西部世界里，一个"人类"的语言可以定义为：
human = """
human = 自己 寻找 活动
自己 = 我 | 我们 
寻找 = 找找 | 想找点 
活动 = 乐子 | 玩的 | 娱乐
"""

# 一个“接待员”的语言可以定义为
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好, | 您好,
询问 = 请问你要 | 您需要
业务相关 = 玩玩 具体业务
玩玩 = null
具体业务 = 喝酒 | 打牌 | 打猎 | 赌博
结尾 = 吗？
"""

for i in range(5):
    print(generate(gram=create_grammar(host, split='='), target='host'))
    print(generate(gram=create_grammar(human, split='='), target='human'))

你好,我是135号,请问你要喝酒吗？
我找找娱乐
您好,我是9号,您需要喝酒吗？
我们想找点娱乐
你好,我是6号,请问你要赌博吗？
我想找点乐子
先生,您好,我是7号,您需要喝酒吗？
我们想找点乐子
你好,我是79号,您需要喝酒吗？
我们想找点乐子


In [7]:
#### 2. 模拟客服机器人
service_robot = """
ask = 打招呼 , 自我介绍 , 询问 

打招呼 = 称谓 , 招呼 
称谓 = 亲 | 先生 | 女士 | 小朋友 | 美女| 帅哥
人称 = 你 | 您 
招呼 = 人称 好

自我介绍 = 我是 数字 号客服|我是客服 名称
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
名称 = 小芳 | 小静 | 张三 | 王五

询问 = 人称 动作 结尾
动作 = 想知道 业务+ | 想了解 业务+ |想咨询 业务+ | 对 业务+ 感兴趣
业务+ = 业务 | 业务 和 业务+ 
业务 = 产品详情 | 购买须知 | 优惠券 | 使用方法 | 订单查询 | 产品维修 | 退货流程 | 退款详情 | 人工客服 
结尾 = 吗？ 
"""

def create_grammar(grammar_str, split='=', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip():
            continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar
example_gram = create_grammar(service_robot)
example_gram

{'ask': [['打招呼', ',', '自我介绍', ',', '询问']],
 '打招呼': [['称谓', ',', '招呼']],
 '称谓': [['亲'], ['先生'], ['女士'], ['小朋友'], ['美女'], ['帅哥']],
 '人称': [['你'], ['您']],
 '招呼': [['人称', '好']],
 '自我介绍': [['我是', '数字', '号客服'], ['我是客服', '名称']],
 '数字': [['单个数字'], ['数字', '单个数字']],
 '单个数字': [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9']],
 '名称': [['小芳'], ['小静'], ['张三'], ['王五']],
 '询问': [['人称', '动作', '结尾']],
 '动作': [['想知道', '业务+'], ['想了解', '业务+'], ['想咨询', '业务+'], ['对', '业务+', '感兴趣']],
 '业务+': [['业务'], ['业务', '和', '业务+']],
 '业务': [['产品详情'],
  ['购买须知'],
  ['优惠券'],
  ['使用方法'],
  ['订单查询'],
  ['产品维修'],
  ['退货流程'],
  ['退款详情'],
  ['人工客服']],
 '结尾': [['吗？']]}

In [8]:
import random
choice = random.choice

def generate(gram, target):
    if target not in gram: return target
    expaned = [generate(gram, t) for t in choice(gram[target])]
    return "".join([e if e != '/n' else '\n' for e in expaned if e != 'null'])

generate(gram=example_gram, target='ask')

'女士,你好,我是客服王五,你想了解退货流程吗？'

In [9]:
# 生成多个句子
def generate_n(n, gram, target):
    return [generate(gram, target) for _ in range(n)]
generate_n(3, gram=example_gram, target='ask')

['美女,你好,我是客服小芳,您想了解使用方法和购买须知和退款详情吗？',
 '女士,您好,我是客服小静,您想咨询退款详情和退货流程和购买须知和人工客服吗？',
 '美女,你好,我是客服王五,您对退款详情和使用方法感兴趣吗？']

In [19]:
# 1.2 基于模板

# 判断是否为变量：变量定义为以 ？开头 的单词

def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

is_variable('?hello'), is_variable('?hello world')

(True, False)

In [20]:
# 判断句子与模板是否匹配，模板中包含上述的变量

def pat_match_1(pattern, sent):
    if is_variable(pattern[0]):
        return True
    else:
        if pattern[0] != sent[0]:
            return False
        else:
            return pat_match_1(pattern[1:], sent[1:])
          
print(pat_match_1("I want ?x", "I want holiday"))
print(pat_match_1("I have dreamed a ?x", "I dreamed about dog"))

True
False


In [21]:
# 判断句子与模板是否匹配，获得匹配的变量
def pat_match_2(pattern, sent):
    if is_variable(pattern[0]):
        return pattern[0], sent[0]
    else:
        if pattern[0] != sent[0]: return False
        else:
            return pat_match_2(pattern[1:], sent[1:])

pattern = 'I want ?X'.split()
sent = "I want holiday".split()
pat_match_2(pattern, sent)

('?X', 'holiday')

In [2]:
# "I need iPhone" 和 "I need ?X" 可以匹配，
# 但是"I need an iPhone" 和 "I need ?X" 就不匹配
# 新建一个变量类型 "?*X", 多了一个星号(*),表示匹配多个
# 多个部分匹配

def pat_match(pattern, saying):
    if not pattern or not saying:
        return []
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]:
            return []
        else:
            return pat_match(pattern[1:], saying[1:])


pat_match("?X greater than ?Y".split(), "3 greater than 2".split())

NameError: name 'is_variable' is not defined

In [45]:
# 将模板中的变量替换成单词，生成句子

def pat_to_dict(patterns):
    return {k: v for k, v in patterns}

def subsitite(rule, parsed_rules):
    if not rule:
        return []
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(
        rule[1:], parsed_rules)

got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
got_patterns

[('?X', 'iPhone')]

In [1]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
' '.join(
    subsitite("What if you mean if you got a ?X".split(),
              pat_to_dict(got_patterns)))
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

NameError: name 'pat_match' is not defined